# FeatherFace Baseline Training and Evaluation

This notebook reproduces the original FeatherFace training process following the author's instructions.

## Overview
- Model: FeatherFace with MobileNetV1 0.25x backbone
- Dataset: WIDERFace (auto-download)
- Expected Results: 0.49M parameters, 90.8% mAP
- Uses original training scripts for faithful reproduction

## 1. Installation and Environment Setup

In [ ]:
# Setup paths - all paths are relative to the FeatherFace root directory
import os
import sys
from pathlib import Path

# Get the project root directory (parent of notebooks/)
PROJECT_ROOT = Path(os.path.abspath('..'))
print(f"Project root: {PROJECT_ROOT}")

# Change to project root for all operations
os.chdir(PROJECT_ROOT)
print(f"Working directory: {os.getcwd()}")

In [ ]:
# Install project in editable mode
!pip install -e .

# Verify imports work
try:
    from models.retinaface import RetinaFace
    from data import cfg_mnet, WiderFaceDetection
    print("✓ Imports successful")
except ImportError as e:
    print(f"✗ Import error: {e}")

## 2. Dataset Download and Preparation

The dataset will be automatically downloaded when training starts. But we can prepare the directories.

In [ ]:
# Check and create data directories
import os
from pathlib import Path

# Create necessary directories
data_dir = Path('data/widerface')
weights_dir = Path('weights')
results_dir = Path('results')

for dir_path in [data_dir, weights_dir, results_dir]:
    dir_path.mkdir(parents=True, exist_ok=True)
    print(f"✓ Directory ready: {dir_path}")

## 3. Download Pre-trained Weights

The model requires pre-trained MobileNetV1 0.25x weights.

In [ ]:
# Pre-trained weights info
PRETRAIN_FILENAME = 'mobilenetV1X0.25_pretrain.tar'
pretrain_path = weights_dir / PRETRAIN_FILENAME

print("=== Pre-trained Weights Download Instructions ===")
print(f"\nWeights should be placed at: {pretrain_path.absolute()}")
print("\nDownload from:")
print("https://drive.google.com/open?id=1oZRSG0ZegbVkVwUd8wUIQx8W7yfZ_ki1")
print(f"\nSave as: {pretrain_path.relative_to('.')}")

if pretrain_path.exists():
    print(f"\n✓ Pre-trained weights found: {pretrain_path.relative_to('.')}")
else:
    print(f"\n✗ Pre-trained weights not found. Please download manually.")

## 4. Model Configuration and Training Parameters

In [ ]:
# Training parameters from original repository
TRAIN_CONFIG = {
    'network': 'mobile0.25',
    'num_workers': 1,  # Adjust based on your system
    'lr': 1e-3,
    'momentum': 0.9,
    'save_folder': 'weights/',
    'resume_net': None,  # Will be set to pretrained weights
    'batch_size': 32,
    'num_epoch': 250,
    'gpu': True,
    'ngpu': 1,
    'pretrain': True
}

print("Training Configuration:")
for key, value in TRAIN_CONFIG.items():
    print(f"  {key}: {value}")